# Introducción
El trastorno por déficit de atención con hiperactividad (TDAH) se caracteriza por la actividad motora excesiva, la falta de atención y la impulsividad. HTR1B es un receptor de serotonina. Los polimorfismos del gen HTR1B se han relacionado con diversos trastornos psiquiátricos, como el trastorno por déficit de atención con hiperactividad (TDAH) y el trastorno obsesivo-compulsivo (TOC).

In [964]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_score

# Dataset description

Each excel sheet, except the first one, have a table with data for each molecular marker. 

First, I will merge all the markers keeping only the records that have all the markers available. This way we can train the logistic model taking into account the other marker for better identification of strong influences in TDAH diagnostic. 

In [965]:
file = "~/PiconCossio/data_science_class/DATOS-TDAH-R_original.xlsx"
xls = pd.ExcelFile(file)
sheets_dict = pd.read_excel(file, sheet_name = xls.sheet_names[1:])

In [966]:
sheets_dict

{'1.HTR1B':     CODIGO  1.HTR1B  TDAH  Edad  Genero  NSE  Adversidad
 0    HH006        2     1     8       2    1           0
 1    HH009        2     1     9       2    3           1
 2    HH014        2     1    15       2    3           1
 3    HH015        1     1    12       1    2           1
 4    HH016        1     0    18       2    3           0
 ..     ...      ...   ...   ...     ...  ...         ...
 388  HP089        1     1    46       1    6           0
 389  HP205        1     0    43       1    4           0
 390  HP211        3     1    46       1    5           0
 391  HP216        2     0    51       1    4           0
 392  HP415        2     0    51       1    2           0
 
 [393 rows x 7 columns],
 '2.HTR2A':     CODIGO  2.HTR2A  TDAH  Edad  Genero  NSE  Adversidad
 0    HH015        2     1    12       1    2           1
 1    HH016        1     0    18       2    3           0
 2    HH022        2     1    15       1    2           1
 3    HH023        3   

In [967]:
for idx, val in enumerate(sheets_dict.keys()):
    if idx == 0:
        df = sheets_dict[val]
        df = pd.concat([df.iloc[:, :3], df.iloc[:,4]], axis=1) ## keeping only CODIGO, SNP, TDAH and Genero
        continue
    df_2 = sheets_dict[val].rename(columns={"CODIGO ": "CODIGO"})
    df = df.merge(df_2.iloc[:,:2], on="CODIGO", how="inner")

## Use the classic 0 for female and 1 for male
df.replace({"Genero":2}, 0, inplace=True)

In [968]:
print(df.shape)
df.head()

(92, 14)


,CODIGO,1.HTR1B,TDAH,Genero,2.HTR2A,3.DRD2,4.DRD4,5. DAT1,6.SLC6A4(LS),7.SLC6A4(VNTR),8.HTR2C,9.SNAP25(DdeI),10.SNP25( Mn1I),11. TPH2
0,HH016,1,0,0,1,2,7,5,2,3,3,2,3,1
1,HH022,2,1,1,2,2,10,4,1,3,5,2,3,1
2,HH024,1,0,0,1,3,7,5,1,3,2,2,3,1
3,HH027,1,0,1,1,1,13,3,1,1,5,1,2,1
4,HH028,1,1,1,2,1,2,5,2,2,5,1,2,2


# First stadistical insights

I do not take into accound *Edad, NSE, Adversidad*, because those are variables that will introduce a lot of noise, hidding the SNP influence and providing false positives, for example, the **6 to 20 years** age range has more data, which is will take a leading role in our model, but actually it is kind of trivial since the TDAH is a condition that appears at early age.

In [969]:
# Existe una mayoria de registros para el genero femenino
df["Genero"].value_counts()

Genero
0    58
1    34
Name: count, dtype: int64

In [970]:
## Existen mas casos de TDAH para el femenino?
## Curiosamente no, hay mayor numero de casos positivos para el genero masculino
df.groupby("Genero")["TDAH"].sum()

Genero
0    14
1    24
Name: TDAH, dtype: int64

Similary, we find that there is an **unbalance** between females and males. We found that **females** got a **higher** number of data and that **TDAH positive represents the minority**, whereas for **males** we observe the opposite where **TDAH positive represents the majority**. Therefore, *Genero* would also take a leading role, and it likely to produce overfitting resulting in a positive diagnosis every time men are recorded. Therefore, for this first analyses *Genero* and *8.HTR2C* will be filtered out.

In the diagonal of the below pairplot, we can see differences in data collection per SNP.

In [971]:
#sns.pairplot(df, kind="kde", diag_kind="kde", corner=True)

In [972]:
df_autosomal = df.drop(["Genero", "8.HTR2C"], axis=1)

# Regresion logistica

All marker possess more than 2 SNP, therefore it is necessary to transform it to binary so the model does no interpret the categories with an ascending meaning. I will use one-hot encoding

In [973]:
df_autosomal.columns

Index(['CODIGO', '1.HTR1B', 'TDAH', '2.HTR2A', '3.DRD2', '4.DRD4', '5. DAT1',
       '6.SLC6A4(LS)', '7.SLC6A4(VNTR)', '9.SNAP25(DdeI)', '10.SNP25( Mn1I)',
       '11. TPH2'],
      dtype='object')

In [974]:
columns=(df_autosomal.columns[1:2]).append(df_autosomal.columns[3:])
columns

Index(['1.HTR1B', '2.HTR2A', '3.DRD2', '4.DRD4', '5. DAT1', '6.SLC6A4(LS)',
       '7.SLC6A4(VNTR)', '9.SNAP25(DdeI)', '10.SNP25( Mn1I)', '11. TPH2'],
      dtype='object')

In [975]:
## Preproccesing
### La columna para el marcador molecular es una variable categorica, que emplea valores numericos (1,2,3). Emplearemos one-hot coding para evitar malinterpretación de valores categoricos.
df_autosomal_transformed = pd.get_dummies(df_autosomal, columns=(df_autosomal.columns[1:2]).append(df_autosomal.columns[3:]))
df_autosomal_transformed.head()

,CODIGO,TDAH,1.HTR1B_1,1.HTR1B_2,1.HTR1B_3,2.HTR2A_1,2.HTR2A_2,2.HTR2A_3,3.DRD2_1,3.DRD2_2,...,7.SLC6A4(VNTR)_3,9.SNAP25(DdeI)_1,9.SNAP25(DdeI)_2,9.SNAP25(DdeI)_3,10.SNP25( Mn1I)_1,10.SNP25( Mn1I)_2,10.SNP25( Mn1I)_3,11. TPH2_1,11. TPH2_2,11. TPH2_3
0,HH016,0,True,False,False,True,False,False,False,True,...,True,False,True,False,False,False,True,True,False,False
1,HH022,1,False,True,False,False,True,False,False,True,...,True,False,True,False,False,False,True,True,False,False
2,HH024,0,True,False,False,True,False,False,False,False,...,True,False,True,False,False,False,True,True,False,False
3,HH027,0,True,False,False,True,False,False,True,False,...,False,True,False,False,False,True,False,True,False,False
4,HH028,1,True,False,False,False,True,False,True,False,...,False,True,False,False,False,True,False,False,True,False


**Note** the increase in the number of columns, it could be a problem for the logistic regression, a PCA could help in the analyses of this with the other meta-variables. However, it does not allow us to check what SNPs might have the strongest influence.

### **Variable Correlation**

Checking the **Pearson** correlaction between SNPs and TDAH diagnosis with SNPs could be useful. Additionally, we can check colleniarity between SNPs. Here, correlation is filtered for values x >= 0.2 or x <= 2

In [976]:
## Lets plot only those with a correlation higher than 0.2 or lower than -0.2
corr_threshold = 0.2
correlation_df = df_autosomal_transformed.iloc[:,1:].corr().where((df_autosomal_transformed.iloc[:,1:].corr() >= corr_threshold) | (df_autosomal_transformed.iloc[:,1:].corr() <= -1*(corr_threshold)))


In [977]:
import plotly.express as px
fig = px.imshow(correlation_df,width=1000, height=800)
fig.show()

The TDAH column reveal some apparently strong positive and negative correlactions. However, it must be visualized taking into account the next histogram. Observe that some SNPs do not have a lot of data, so any significative correlation could be despicable.   

In [978]:
data_per_snp = pd.DataFrame(df_autosomal_transformed.iloc[:,2:].sum())
data_per_snp.reset_index(inplace=True)
data_per_snp.columns=["SNP","Count"]
fig = px.histogram(x=data_per_snp["SNP"], y=data_per_snp["Count"])
fig.show()

In [979]:
data_per_snp_filt = data_per_snp[data_per_snp["Count"]>=10]
snp_passed = data_per_snp_filt["SNP"].to_list()
snp_passed[:0] = ["CODIGO", "TDAH"]

# Train logistic model

Here, we will use cross validation, due to the scarcity of data.

### Only autosomal SNPs

As it was said before, due to the differences in number of data between Genre, we are using only **autosomal markers**.

In [980]:
df_autosomal_transformed = df_autosomal_transformed[snp_passed]
df_autosomal_transformed.head()

,CODIGO,TDAH,1.HTR1B_1,1.HTR1B_2,1.HTR1B_3,2.HTR2A_1,2.HTR2A_2,2.HTR2A_3,3.DRD2_1,3.DRD2_2,...,7.SLC6A4(VNTR)_1,7.SLC6A4(VNTR)_2,7.SLC6A4(VNTR)_3,9.SNAP25(DdeI)_1,9.SNAP25(DdeI)_2,10.SNP25( Mn1I)_1,10.SNP25( Mn1I)_2,10.SNP25( Mn1I)_3,11. TPH2_1,11. TPH2_2
0,HH016,0,True,False,False,True,False,False,False,True,...,False,False,True,False,True,False,False,True,True,False
1,HH022,1,False,True,False,False,True,False,False,True,...,False,False,True,False,True,False,False,True,True,False
2,HH024,0,True,False,False,True,False,False,False,False,...,False,False,True,False,True,False,False,True,True,False
3,HH027,0,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,True,False
4,HH028,1,True,False,False,False,True,False,True,False,...,False,True,False,True,False,False,True,False,False,True


In [981]:
## Model set up
model_autosomal = LogisticRegression(fit_intercept=False, penalty="elasticnet", solver="saga", l1_ratio=0.3)

In [982]:
## Cross validation
score = cross_val_score(model_autosomal, df_autosomal_transformed.iloc[:,2:] ,df_autosomal_transformed["TDAH"].values, cv=5, scoring='recall_weighted')
print("score in each iteration: ",score)
print("Mean and std score by cross validation: ", score.mean(), score.std())

score in each iteration:  [0.47368421 0.57894737 0.55555556 0.66666667 0.44444444]
Mean and std score by cross validation:  0.543859649122807 0.079066403654529


In [983]:
model_autosomal.fit(df_autosomal_transformed.iloc[:,2:], df_autosomal_transformed["TDAH"].values)

LogisticRegression(fit_intercept=False, l1_ratio=0.3, penalty='elasticnet',
                   solver='saga')

### Coefficients

Once the model is trained we can get the coefficients which will tell us what are the features (SNPs) with the highest influence in the model. Consider that these coefficients are the same as those that are producing the great F1 score showed by cross-validation. Additionally, here we are using elasticnet this method allows us to configure a balance between L1 and L2 regularization so, we will see some coefficients in zero that could be consider as with no correlation so far (consider also some of them are 0 due to the amount of data), L1 can be kind of agressive, so will be important to achieve a equilibrium, in this case I will use a ratio of 0.3

In [984]:
feature_coefficients = pd.DataFrame({
    'Feature': df_autosomal_transformed.iloc[:,2:].columns,
    'Coefficient': model_autosomal.coef_[0]
})

fig = px.scatter(feature_coefficients, x=feature_coefficients["Feature"], y=feature_coefficients["Coefficient"])
fig.update_traces(marker={'size': 10})

In [985]:
#snp_significative = feature_coefficients[(feature_coefficients["Coefficient"]>0) | (feature_coefficients["Coefficient"]<0)]["Feature"].to_list()
snp_significative = feature_coefficients[(feature_coefficients["Coefficient"]>0)]["Feature"].to_list()
snp_significative

['1.HTR1B_2',
 '3.DRD2_2',
 '7.SLC6A4(VNTR)_2',
 '9.SNAP25(DdeI)_2',
 '10.SNP25( Mn1I)_1',
 '11. TPH2_1']

### Complementary

Additionally, the same logistic model but from statsmodels will be used. This model have a small advantage, that produces P-values for features which are commonly used in biology. However, it presents problems with a lot of features like in this case, therefore I will only use those SNPs that presented a coeficient > 0

In [986]:
import statsmodels.api as sm

model_autosomal_sm=sm.Logit(df_autosomal_transformed["TDAH"].values[:], df_autosomal_transformed[snp_significative])

In [987]:
result = model_autosomal_sm.fit_regularized()

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6701058684277066
            Iterations: 25
            Function evaluations: 25
            Gradient evaluations: 25


In [988]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   92
Model:                          Logit   Df Residuals:                       86
Method:                           MLE   Df Model:                            5
Date:                Fri, 07 Jun 2024   Pseudo R-squ.:                 0.01157
Time:                        14:14:55   Log-Likelihood:                -61.650
converged:                       True   LL-Null:                       -62.371
Covariance Type:            nonrobust   LLR p-value:                    0.9196
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
1.HTR1B_2             0.5299      0.405      1.308      0.191      -0.264       1.324
3.DRD2_2             -0.3832      0.432     -0.886      0.375      -1.231       0.464
7.SLC6A4(VNTR)_2      0.1944      0.371      0.523      0.601      -0.534       0.923
9.SNAP25(DdeI)_2     -0.4008      0.431     -0.930      0.352      -1.245       0.444
10.SNP25( Mn1I)_1    -0.0097      0.574     -0.017      0.987      -1.135       1.116
11. TPH2_1           -0.1908      0.404     -0.472      0.637      -0.983       0.601
=====================================================================================
"""

# Only SNPs in males

Here, I will use only males.

In [989]:
df.head()

,CODIGO,1.HTR1B,TDAH,Genero,2.HTR2A,3.DRD2,4.DRD4,5. DAT1,6.SLC6A4(LS),7.SLC6A4(VNTR),8.HTR2C,9.SNAP25(DdeI),10.SNP25( Mn1I),11. TPH2
0,HH016,1,0,0,1,2,7,5,2,3,3,2,3,1
1,HH022,2,1,1,2,2,10,4,1,3,5,2,3,1
2,HH024,1,0,0,1,3,7,5,1,3,2,2,3,1
3,HH027,1,0,1,1,1,13,3,1,1,5,1,2,1
4,HH028,1,1,1,2,1,2,5,2,2,5,1,2,2


In [990]:
only_males = df[df["Genero"] == 1]
only_males.shape

(34, 14)

In [991]:
only_males.head()

,CODIGO,1.HTR1B,TDAH,Genero,2.HTR2A,3.DRD2,4.DRD4,5. DAT1,6.SLC6A4(LS),7.SLC6A4(VNTR),8.HTR2C,9.SNAP25(DdeI),10.SNP25( Mn1I),11. TPH2
1,HH022,2,1,1,2,2,10,4,1,3,5,2,3,1
3,HH027,1,0,1,1,1,13,3,1,1,5,1,2,1
4,HH028,1,1,1,2,1,2,5,2,2,5,1,2,2
5,HH031,2,0,1,2,1,9,4,2,2,5,1,3,2
6,HH057,1,0,1,1,1,2,4,2,2,5,1,2,2


In [992]:
## Preproccesing
### La columna para el marcador molecular es una variable categorica, que emplea valores numericos (1,2,3). Emplearemos one-hot coding para evitar malinterpretación de valores categoricos.
df_males_transformed = pd.get_dummies(only_males, columns=(only_males.columns[1:2]).append(only_males.columns[4:]))
df_males_transformed.head()

,CODIGO,TDAH,Genero,1.HTR1B_1,1.HTR1B_2,1.HTR1B_3,2.HTR2A_1,2.HTR2A_2,2.HTR2A_3,3.DRD2_1,...,8.HTR2C_5,9.SNAP25(DdeI)_1,9.SNAP25(DdeI)_2,9.SNAP25(DdeI)_3,10.SNP25( Mn1I)_1,10.SNP25( Mn1I)_2,10.SNP25( Mn1I)_3,11. TPH2_1,11. TPH2_2,11. TPH2_3
1,HH022,1,1,False,True,False,False,True,False,False,...,True,False,True,False,False,False,True,True,False,False
3,HH027,0,1,True,False,False,True,False,False,True,...,True,True,False,False,False,True,False,True,False,False
4,HH028,1,1,True,False,False,False,True,False,True,...,True,True,False,False,False,True,False,False,True,False
5,HH031,0,1,False,True,False,False,True,False,True,...,True,True,False,False,False,False,True,False,True,False
6,HH057,0,1,True,False,False,True,False,False,True,...,True,True,False,False,False,True,False,False,True,False


In [993]:
data_per_snp = pd.DataFrame(df_males_transformed.iloc[:,3:].sum())
data_per_snp.reset_index(inplace=True)
data_per_snp.columns=["SNP","Count"]
fig = px.histogram(x=data_per_snp["SNP"], y=data_per_snp["Count"])
fig.show()

In [994]:
data_per_snp_filt = data_per_snp[data_per_snp["Count"]>=5]
snp_passed = data_per_snp_filt["SNP"].to_list()
snp_passed[:0] = ["CODIGO", "TDAH", "Genero"]

In [995]:
## Model set up
model_males = LogisticRegression(fit_intercept=False, penalty="elasticnet", solver="saga", l1_ratio=0.3)

In [996]:
##  cross validation
score = cross_val_score(model_males, df_males_transformed[snp_passed].iloc[:,3:], df_males_transformed["TDAH"].values, cv=5, scoring="recall_weighted")
print("score in each iteration: ",score)
print("Mean and std score by cross validation: ", score.mean(), score.std())

score in each iteration:  [0.71428571 0.71428571 0.71428571 0.42857143 0.66666667]
Mean and std score by cross validation:  0.6476190476190475 0.11106575037800574


In [997]:
model_males.fit(df_males_transformed[snp_passed].iloc[:,3:], df_males_transformed["TDAH"].values)

LogisticRegression(fit_intercept=False, l1_ratio=0.3, penalty='elasticnet',
                   solver='saga')

In [998]:
feature_coefficients = pd.DataFrame({
    'Feature': df_males_transformed[snp_passed].iloc[:,3:].columns,
    'Coefficient': model_males.coef_[0]
})

fig = px.scatter(feature_coefficients, x=feature_coefficients["Feature"], y=feature_coefficients["Coefficient"])
fig.update_traces(marker={'size': 10})

In [999]:
print(classification_report(df_males_transformed["TDAH"].values, model_males.predict(df_males_transformed[snp_passed].iloc[:,3:])))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75        10
           1       0.86      1.00      0.92        24

    accuracy                           0.88        34
   macro avg       0.93      0.80      0.84        34
weighted avg       0.90      0.88      0.87        34



In [1000]:
#snp_significative = feature_coefficients[(feature_coefficients["Coefficient"]>0) | (feature_coefficients["Coefficient"]<0)]["Feature"].to_list()
snp_significative = feature_coefficients[(feature_coefficients["Coefficient"]>0)]["Feature"].to_list()
snp_significative

['1.HTR1B_2',
 '2.HTR2A_3',
 '3.DRD2_2',
 '4.DRD4_10',
 '5. DAT1_5',
 '6.SLC6A4(LS)_3',
 '7.SLC6A4(VNTR)_2',
 '8.HTR2C_4',
 '9.SNAP25(DdeI)_2',
 '10.SNP25( Mn1I)_1',
 '10.SNP25( Mn1I)_3',
 '11. TPH2_2']

### Complementary
Additionally, the same logistic model but from statsmodels will be used. This model have a small advantage, that produces P-values for features which are commonly used in biology. However, it presents problems with a lot of features like in this case, therefore I will only use those SNPs that presented a coeficient > 0

In [1001]:
model_autosomal_sm=sm.Logit(df_males_transformed["TDAH"].values[:], df_males_transformed[snp_significative])
result = model_autosomal_sm.fit()
result.summary()

         Current function value: 0.393453
         Iterations: 35


/home/genmol1/miniforge3/envs/data_science/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   34
Model:                          Logit   Df Residuals:                       22
Method:                           MLE   Df Model:                           11
Date:                Fri, 07 Jun 2024   Pseudo R-squ.:                  0.3505
Time:                        14:14:56   Log-Likelihood:                -13.377
converged:                      False   LL-Null:                       -20.597
Covariance Type:            nonrobust   LLR p-value:                    0.2096
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
1.HTR1B_2             0.1121      1.050      0.107      0.915      -1.946       2.171
2.HTR2A_3             1.1032      1.639      0.673      0.501      -2.110       4.316
3.DRD2_2             27.8448   4.94e+05   5.64e-05      1.000   -9.68e+05    9.68e+05
4.DRD4_10            -0.4895      1.121     -0.437      0.662      -2.687       1.708
5. DAT1_5             0.4859      1.195      0.407      0.684      -1.857       2.829
6.SLC6A4(LS)_3        0.6467      1.393      0.464      0.642      -2.084       3.377
7.SLC6A4(VNTR)_2      0.2111      1.190      0.177      0.859      -2.122       2.544
8.HTR2C_4            22.8679   5.05e+04      0.000      1.000   -9.89e+04    9.89e+04
9.SNAP25(DdeI)_2     -0.6893      1.153     -0.598      0.550      -2.948       1.570
10.SNP25( Mn1I)_1     0.9055      1.326      0.683      0.495      -1.694       3.505
10.SNP25( Mn1I)_3     0.3881      1.255      0.309      0.757      -2.072       2.848
11. TPH2_2            0.0748      1.161      0.064      0.949      -2.202       2.351
=====================================================================================

Possibly complete quasi-separation: A fraction 0.32 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

# Only females

In [1002]:
df.head()

,CODIGO,1.HTR1B,TDAH,Genero,2.HTR2A,3.DRD2,4.DRD4,5. DAT1,6.SLC6A4(LS),7.SLC6A4(VNTR),8.HTR2C,9.SNAP25(DdeI),10.SNP25( Mn1I),11. TPH2
0,HH016,1,0,0,1,2,7,5,2,3,3,2,3,1
1,HH022,2,1,1,2,2,10,4,1,3,5,2,3,1
2,HH024,1,0,0,1,3,7,5,1,3,2,2,3,1
3,HH027,1,0,1,1,1,13,3,1,1,5,1,2,1
4,HH028,1,1,1,2,1,2,5,2,2,5,1,2,2


In [1003]:
only_females = df[df["Genero"] == 0]
only_females.shape

(58, 14)

In [1004]:
only_females.head()

,CODIGO,1.HTR1B,TDAH,Genero,2.HTR2A,3.DRD2,4.DRD4,5. DAT1,6.SLC6A4(LS),7.SLC6A4(VNTR),8.HTR2C,9.SNAP25(DdeI),10.SNP25( Mn1I),11. TPH2
0,HH016,1,0,0,1,2,7,5,2,3,3,2,3,1
2,HH024,1,0,0,1,3,7,5,1,3,2,2,3,1
7,HH060,1,0,0,2,1,3,2,1,2,2,1,3,1
8,HH063,2,1,0,1,1,7,5,2,2,3,2,3,2
10,HH073,2,0,0,1,3,7,4,2,2,3,1,1,2


In [1005]:
## Preproccesing
### La columna para el marcador molecular es una variable categorica, que emplea valores numericos (1,2,3). Emplearemos one-hot coding para evitar malinterpretación de valores categoricos.
df_females_transformed = pd.get_dummies(only_females, columns=(only_females.columns[1:2]).append(only_females.columns[4:]))
df_females_transformed.head()

,CODIGO,TDAH,Genero,1.HTR1B_1,1.HTR1B_2,1.HTR1B_3,2.HTR2A_1,2.HTR2A_2,2.HTR2A_3,3.DRD2_1,...,8.HTR2C_3,9.SNAP25(DdeI)_1,9.SNAP25(DdeI)_2,9.SNAP25(DdeI)_3,10.SNP25( Mn1I)_1,10.SNP25( Mn1I)_2,10.SNP25( Mn1I)_3,11. TPH2_1,11. TPH2_2,11. TPH2_3
0,HH016,0,0,True,False,False,True,False,False,False,...,True,False,True,False,False,False,True,True,False,False
2,HH024,0,0,True,False,False,True,False,False,False,...,False,False,True,False,False,False,True,True,False,False
7,HH060,0,0,True,False,False,False,True,False,True,...,False,True,False,False,False,False,True,True,False,False
8,HH063,1,0,False,True,False,True,False,False,True,...,True,False,True,False,False,False,True,False,True,False
10,HH073,0,0,False,True,False,True,False,False,False,...,True,True,False,False,True,False,False,False,True,False


In [1006]:
data_per_snp = pd.DataFrame(df_females_transformed.iloc[:,3:].sum())
data_per_snp.reset_index(inplace=True)
data_per_snp.columns=["SNP","Count"]
fig = px.histogram(x=data_per_snp["SNP"], y=data_per_snp["Count"])
fig.show()

In [1007]:
data_per_snp_filt = data_per_snp[data_per_snp["Count"]>=10]
snp_passed = data_per_snp_filt["SNP"].to_list()
snp_passed[:0] = ["CODIGO", "TDAH", "Genero"]

In [1008]:
## Model set up
model_females = LogisticRegression(fit_intercept=False, penalty="elasticnet", solver="saga", l1_ratio=0.3)

In [1009]:
##  cross validation
score = cross_val_score(model_females, df_females_transformed[snp_passed].iloc[:,3:], df_females_transformed["TDAH"].values, cv=5, scoring="recall_weighted")
print("score in each iteration: ",score)
print("Mean and std score by cross validation: ", score.mean(), score.std())

score in each iteration:  [0.66666667 0.75       0.75       0.81818182 0.72727273]
Mean and std score by cross validation:  0.7424242424242424 0.048626686472367335


In [1010]:
model_females.fit(df_females_transformed[snp_passed].iloc[:,3:], df_females_transformed["TDAH"].values)

LogisticRegression(fit_intercept=False, l1_ratio=0.3, penalty='elasticnet',
                   solver='saga')

In [1011]:
feature_coefficients = pd.DataFrame({
    'Feature': df_females_transformed[snp_passed].iloc[:,3:].columns,
    'Coefficient': model_females.coef_[0]
})

fig = px.scatter(feature_coefficients, x=feature_coefficients["Feature"], y=feature_coefficients["Coefficient"])
fig.update_traces(marker={'size': 10})

In [1012]:
print(classification_report(df_females_transformed["TDAH"].values, model_females.predict(df_females_transformed[snp_passed].iloc[:,3:])))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        44
           1       1.00      0.14      0.25        14

    accuracy                           0.79        58
   macro avg       0.89      0.57      0.56        58
weighted avg       0.84      0.79      0.73        58



In [1013]:
#snp_significative = feature_coefficients[(feature_coefficients["Coefficient"]>0) | (feature_coefficients["Coefficient"]<0)]["Feature"].to_list()
snp_significative = feature_coefficients[(feature_coefficients["Coefficient"]>0)]["Feature"].to_list()
snp_significative

['1.HTR1B_2', '3.DRD2_2', '4.DRD4_7', '7.SLC6A4(VNTR)_2', '11. TPH2_1']

### Complementary
Additionally, the same logistic model but from statsmodels will be used. This model have a small advantage, that produces P-values for features which are commonly used in biology. However, it presents problems with a lot of features like in this case, therefore I will only use those SNPs that presented a coeficient > 0

In [1014]:
model_autosomal_sm=sm.Logit(df_females_transformed["TDAH"].values[:], df_females_transformed[snp_significative])
result = model_autosomal_sm.fit(method="lbfgs", maxiter=100)
result.summary()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  9.48276D-02

At iterate    1    f=  6.37288D-01    |proj g|=  3.62980D-02

At iterate    2    f=  6.29028D-01    |proj g|=  1.37356D-02

At iterate    3    f=  6.28074D-01    |proj g|=  4.73346D-03

At iterate    4    f=  6.27623D-01    |proj g|=  3.97456D-03

At iterate    5    f=  6.27398D-01    |proj g|=  6.89640D-04

At iterate    6    f=  6.27394D-01    |proj g|=  7.60035D-05

At iterate    7    f=  6.27394D-01    |proj g|=  2.41235D-05

At iterate    8    f=  6.27394D-01    |proj g|=  3.33131D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg 

 This problem is unconstrained.


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   58
Model:                          Logit   Df Residuals:                       53
Method:                           MLE   Df Model:                            4
Date:                Fri, 07 Jun 2024   Pseudo R-squ.:                 -0.1352
Time:                        14:14:56   Log-Likelihood:                -36.389
converged:                       True   LL-Null:                       -32.055
Covariance Type:            nonrobust   LLR p-value:                     1.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
1.HTR1B_2           -0.1399      0.561     -0.249      0.803      -1.240       0.960
3.DRD2_2            -0.5203      0.545     -0.954      0.340      -1.589       0.548
4.DRD4_7            -0.3886      0.496     -0.783      0.434      -1.361       0.584
7.SLC6A4(VNTR)_2    -0.4760      0.510     -0.933      0.351      -1.476       0.524
11. TPH2_1          -0.0419      0.546     -0.077      0.939      -1.112       1.028
====================================================================================
"""